In [2]:
# Exploratory Data Analysis (EDA)
# Project: House Price Prediction - ML Pipeline
# Purpose:
# - Understand dataset structure
# - Identify missing values
# - Analyze feature distributions
# - Study relationship between features and house price


In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Plot settings
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


In [4]:
# NOTE:
# Dataset will be loaded from data/raw/ in upcoming steps
# Example:
# df = pd.read_csv("data/raw/house_prices.csv")
